# Load libaries

In [1]:
import matplotlib.pylab as plt
import torch
import pytorch_lightning as pl
import numpy as np

from iterativenn.utils.model_factory import ModelFactory_callback
from iterativenn.utils.data_factory import DataFactory
from iterativenn.utils.logger_factory import LoggerFactory
from iterativenn.lit_modules.IteratedModel import IteratedModel, ConfigCallbacks
from iterativenn.nn_modules.Sequential1D import Sequential1D
from iterativenn.nn_modules.MaskedLinear import MaskedLinear


ModuleNotFoundError: No module named 'gym'

# Introduction

This notebook is intended to be, in as simple a way as possible, a comparison of a classic MLP with an INN. 

1.  This will be a baseline solution that should be familiar to most students.
2.  It will be a sanity check on the MLP and INN solutions.  I.e., they should be able to learn the same thing as the least squares solution, if not better.
3.  It will be a starting place for any student working on similar problems. 


# Top level parameters


In [ ]:
globals = {}
globals['layer_sizes'] = [2500, 500, 200, 10]
globals['activation'] = 'ELU'
globals['accelerator'] = 'auto'
# globals['accelerator'] = 'cpu'
globals['optimizer'] = 'Adam'
globals['learning rate'] = 0.01
globals['bias'] = False
globals['initialization mean'] = 0.0
globals['initialization std'] = 0.01


In [ ]:
globals['do_MLP'] = True
globals['do_sequence_MLP'] = True
globals['do_INN'] = True
globals['do_sparse_INN'] = True
globals['saved models'] = {}

In [ ]:
fast_dev_run = False
if fast_dev_run:
    globals['number_of_runs'] = 1
    globals['data size'] = 16
    globals['batch size'] = 8
    globals['epochs'] = 1
else:
    globals['number_of_runs'] = 3
    globals['data size'] = 1000
    globals['batch size'] = 128
    globals['epochs'] = 3

## MLP data loading

# Data loading

Here we begin by loading the data.  We will ust the classic MNIST data set for these examples.  The iterativenn package provides utilities to make reading in the data easy.

In [ ]:
def MLP_data_loader():
    data_cfg = {
        'dataset': 'MNIST',
        'transform': 'both',
        'train_size': globals['data size'],
        'validation_size': globals['data size'],
        'test_size': globals['data size'],
        'batch_size': globals['batch size'],
    }

    data = DataFactory(data_cfg)
    return data

Each item is a tuple with two entries. The first is an image and the second is a label.  

The image is a 50x50 array of pixels.  Note, this is different from the MNIST standard of 28x28!   We scale the images up so that we can do
transformations on them to make the problem more interesting.

The label is a number between 0 and 9.  The label is the number that the image represents.

In [ ]:
if globals['do_MLP']:
    data = MLP_data_loader()
    sample = next(iter(data.train_dataloader()))
    for i in range(3):
        plt.figure(figsize=(2, 2))
        plt.imshow(sample[0][i, 0, :, :])
        plt.title(f'true category {sample[1][i]}')
        plt.show()

## MLP sequence data loading

In [ ]:
def MLP_sequence_data_loader():
    data_cfg = {
        'dataset': 'MNIST',
        'transform': 'both',
        'train_size': globals['data size'],
        'validation_size': globals['data size'],
        'test_size': globals['data size'],
        'batch_size': globals['batch size'],
        'sequence_dict': {
            'type': 'uniform',
            'min_copies': 1,
            'max_copies': 1,
            'evaluate_loss': 'last'
        }
    }
    data = DataFactory(data_cfg)
    return data

In [ ]:
if globals['do_sequence_MLP']:
    data = MLP_sequence_data_loader()
    sample = next(iter(data.train_dataloader()))
    sample[0]['x']
    for i in range(3):
        copies = len(sample[i]['x'])
        _, ax = plt.subplots(ncols=copies, squeeze=False)
        for j in range(copies):
            ax[0, j].imshow(sample[i]['x'][j][0, :, :])
            ax[0, j].set_title(f'{sample[i]["y"][j]}')

## INN data loading

In [ ]:
def INN_data_loader():
    data_cfg = {
        'dataset': 'MNIST',
        'transform': 'both',
        'train_size': globals['data size'],
        'validation_size': globals['data size'],
        'test_size': globals['data size'],
        'batch_size': globals['batch size'],
        'sequence_dict': {
            'type': 'uniform',
            'min_copies': 3,
            'max_copies': 3,
            'evaluate_loss': 'last'
        }
    }
    data = DataFactory(data_cfg)
    return data

In [ ]:
if globals['do_INN']:
    data = INN_data_loader()
    sample = next(iter(data.train_dataloader()))
    sample[0]['x']
    for i in range(3):
        copies = len(sample[i]['x'])
        _, ax = plt.subplots(ncols=copies)
        for j in range(copies):
            ax[j].imshow(sample[i]['x'][j][0, :, :])
            ax[j].set_title(f'{sample[i]["y"][j]}')

# MLP solution

In [ ]:
class LitModule(pl.LightningModule):
    def __init__(self, module):
        super().__init__()
        self.module = module
        self.losses = []
    
    def forward(self, X):
        return self.module(X)

    def training_step(self, batch, batch_idx):
        X,Y = batch
        Y_hat = self.module(X.view(X.shape[0], -1))
        loss = torch.nn.CrossEntropyLoss(reduction='sum')(Y_hat, Y)
        self.losses.append(loss)
        return loss
    
    def configure_optimizers(self):
        if globals['optimizer'] == 'Adam':
            optimizer = torch.optim.Adam(params=self.parameters(), lr=globals['learning rate'])
        elif globals['optimizer'] == 'SGD':
            return torch.optim.SGD(params=self.parameters(), lr=globals['learning rate'])
        return optimizer

In [ ]:
def train_MLP():
    data = MLP_data_loader()
    # Sequential1D is just a wrapper around nn.Sequential that
    # records the input and output sizes of the network.  This is useful
    # for the INN solution below as the MLP can be used as a block in an INN.
    # However, the as far as this part is concerned, it is identical to nn.Sequential.
    MLP = Sequential1D(in_features=globals['layer_sizes'][0],
                       out_features=globals['layer_sizes'][-1]) 

    for i in range(len(globals['layer_sizes'])-1):
        linear_module = torch.nn.Linear(globals['layer_sizes'][i], globals['layer_sizes'][i+1], bias=globals['bias'])
        with torch.no_grad():
            torch.nn.init.normal_(linear_module.weight, 
                                  globals['initialization mean'], 
                                  globals['initialization std'])
        MLP.add_module(f'linear{i}', linear_module)
        # always end with a linear layer
        if i < len(globals['layer_sizes'])-2:      
            if globals['activation'] == 'ELU':
                MLP.add_module(f'activation{i}', torch.nn.ELU())      
            else:
                raise ValueError(f'Unknown activation {globals["activation"]}')

    MLP_module = LitModule(MLP)

    trainer = pl.Trainer(max_epochs=globals['epochs'],
                         logger=LoggerFactory(),
                        devices=1 if torch.cuda.is_available() else 1,
                        accelerator=globals['accelerator'])
    trainer.fit(MLP_module, data)
    return MLP_module

In [ ]:
if globals['do_MLP']:
    name = 'MLP'
    globals['saved models'][name] = {}
    # run all the models and save them
    globals['saved models'][name]['models'] = [train_MLP() for i in range(globals['number_of_runs'])]
    # save the number of trainable parameters
    globals['saved models'][name]['average trainable params'] = np.mean([sum(p.numel() for p in model.parameters() if p.requires_grad) for model in globals['saved models'][name]['models']])
    # save the losses
    losses = [torch.tensor(model.losses).detach().numpy() for model in globals['saved models'][name]['models']]
    globals['saved models'][name]['average losses'] = np.mean(losses, axis=0)
    globals['saved models'][name]['std losses'] = np.std(losses, axis=0)

In [ ]:
if globals['do_MLP']:
    name = 'MLP'
    plt.errorbar(range(len(globals['saved models'][name]['average losses'])), 
                 globals['saved models'][name]['average losses'], 
                 globals['saved models'][name]['std losses'],
                 capsize=5,
                 label=name)

# MLP using sequence data

In [ ]:
def train_sequence_MLP():
    data = MLP_sequence_data_loader()
    # Sequential1D is just a wrapper around nn.Sequential that
    # records the input and output sizes of the network.  This is useful
    # for the INN solution below as the MLP can be used as a block in an INN.
    # However, the as far as this part is concerned, it is identical to nn.Sequential.
    MLP = Sequential1D(in_features=globals['layer_sizes'][0],
                       out_features=globals['layer_sizes'][-1]) 

    for i in range(len(globals['layer_sizes'])-1):

        # You can use a masked linear layer if you want
        linear_module = MaskedLinear(globals['layer_sizes'][i], 
                                        globals['layer_sizes'][i+1], 
                                        bias=globals['bias'])
        with torch.no_grad():
            torch.nn.init.normal_(linear_module.weight_0, 
                                  globals['initialization mean'], 
                                  globals['initialization std'])

        # Or you can use a regular linear layer, the answers are the same.
        # linear_module = torch.nn.Linear(globals['layer_sizes'][i], 
        #                                 globals['layer_sizes'][i+1], 
        #                                 bias=globals['bias'])
        # with torch.no_grad():
        #     torch.nn.init.normal_(linear_module.weight, 
        #                           globals['initialization mean'], 
        #                           globals['initialization std'])

        MLP.add_module(f'linear{i}', linear_module)
        # always end with a linear layer
        if i < len(globals['layer_sizes'])-2:      
            if globals['activation'] == 'ELU':
                MLP.add_module(f'activation{i}', torch.nn.ELU())      
            else:
                raise ValueError(f'Unknown activation {globals["activation"]}')

    cfg = {
        "callbacks": {
            "loss": {
                "func": "CrossEntropyLoss",
                "idx_list" : range(0, globals['layer_sizes'][-1]),
            },
            "initialization": {
                "func": "zeros",
                "size": globals['layer_sizes'][0],
            },
            "data": {
                "func": "insert",
                "idx_list": range(0, globals['layer_sizes'][0]),
                "flatten_input": True,
            },
            "output": {
                "func": "max",
                "idx_list" : range(0, globals['layer_sizes'][-1]),
            },
        }
    }

    MLP_callbacks = ConfigCallbacks(cfg['callbacks'])

    MLP_module = IteratedModel(MLP, MLP_callbacks, normalize_loss=False, 
                              learning_rate=globals['learning rate'], optimizer=globals['optimizer'])
    trainer = pl.Trainer(max_epochs=globals['epochs'],
                         devices=1 if torch.cuda.is_available() else 1,
                         logger=LoggerFactory(),
                         accelerator=globals['accelerator'])
    trainer.fit(MLP_module, data)
    return MLP_module

In [ ]:
if globals['do_sequence_MLP']:
    name = 'sequence_MLP'
    globals['saved models'][name] = {}
    # run all the models and save them
    globals['saved models'][name]['models'] = [train_sequence_MLP() for i in range(globals['number_of_runs'])]
    # save the number of trainable parameters
    globals['saved models'][name]['average trainable params'] = np.mean([model.number_of_trainable_parameters() for model in globals['saved models'][name]['models']])
    # save the losses
    losses = [torch.tensor(model.losses).detach().numpy() for model in globals['saved models'][name]['models']]
    globals['saved models'][name]['average losses'] = np.mean(losses, axis=0)
    globals['saved models'][name]['std losses'] = np.std(losses, axis=0)

In [ ]:
if globals['do_sequence_MLP']:
    name = 'sequence_MLP'
    plt.errorbar(range(len(globals['saved models'][name]['average losses'])), 
                 globals['saved models'][name]['average losses'], 
                 globals['saved models'][name]['std losses'],
                 capsize=5,
                 label=name)

# INN that simulate MLP

$$ 
\begin{bmatrix}
x & 0 & 0 & 0 
\end{bmatrix}
\begin{bmatrix}
I & W_1 & 0   &  0 \\
0 & 0   & W_2 &  0 \\
0 & 0   & 0   & W_3 \\
0 & 0   & 0   & 0
\end{bmatrix}
=
\begin{bmatrix}
x & W_1 x & 0 & 0
\end{bmatrix}
$$

$$ 
\begin{bmatrix}
x & W_1 x & 0 & 0 
\end{bmatrix}
\begin{bmatrix}
I & W_1 & 0   &  0 \\
0 & 0   & W_2 &  0 \\
0 & 0   & 0   & W_3 \\
0 & 0   & 0   & 0
\end{bmatrix}
=
\begin{bmatrix}
x & W_1 x & W_2 W_1 x & 0
\end{bmatrix}
$$

$$ 
\begin{bmatrix}
x & W_1 x & W_2 W_1 x & 0 
\end{bmatrix}
\begin{bmatrix}
I & W_1 & 0   &  0 \\
0 & 0   & W_2 &  0 \\
0 & 0   & 0   & W_3 \\
0 & 0   & 0   & 0
\end{bmatrix}
=
\begin{bmatrix}
x & W_1 x & W_2 W_1 x & W_3 W_2 W_1 x
\end{bmatrix}
$$

In [ ]:
def train_INN():
    data = INN_data_loader()
    default_module = 'MaskedLinear.from_description'
    default_module_kwargs = {'block_type': 'W', 
                             'initialization_type': f'G={globals["initialization mean"]},{globals["initialization std"]}', 
                             'trainable': True, 'bias': globals['bias']}
    
    cfg = {
        "model_type": "sequential2D",   
        "in_features_list": globals['layer_sizes'], 
        "out_features_list": globals['layer_sizes'], 

        "block_types": [
            ['Identity', default_module, None,           None],
            [None,       None,           default_module, None],
            [None,       None,           None,           default_module],
            [None,       None,           None,          None],
        ],
        "block_kwargs": [
            [None, default_module_kwargs, None, None],
            [None, None, default_module_kwargs, None],
            [None, None, None, default_module_kwargs],
            [None, None, None, None],
        ],

        "activations" : ['Identity', 'ELU', 'ELU', 'Identity'],
        "activation_sizes" : globals['layer_sizes'],
        "callbacks": {
            "loss": {
                "func": "CrossEntropyLoss",
                "idx_list" : range(sum(globals['layer_sizes'][:-1]), sum(globals['layer_sizes'])),
            },
            "initialization": {
                "func": "zeros",
                "size": sum(globals['layer_sizes']),
            },
            "data": {
                "func": "insert",
                "idx_list": range(0, globals['layer_sizes'][0]),
                "flatten_input": True,
            },
            "output": {
                "func": "max",
                "idx_list" : range(sum(globals['layer_sizes'][:-1]), sum(globals['layer_sizes']))
            },
        }
    }
    # This is an IteratedModule, which is a wrapper around a module
    module = ModelFactory_callback(cfg)
    INN_module = IteratedModel(module.model, module.callbacks, normalize_loss=False, 
                            learning_rate=globals['learning rate'], optimizer=globals['optimizer'])
    trainer = pl.Trainer(max_epochs=globals['epochs'],
                         logger=LoggerFactory(),
                         devices=1 if torch.cuda.is_available() else 1,
                         accelerator=globals['accelerator'])
    trainer.fit(INN_module, data)
    return INN_module

In [ ]:
if globals['do_INN']:
    name = 'INN'
    globals['saved models'][name] = {}
    # run all the models and save them
    globals['saved models'][name]['models'] = [train_INN() for i in range(globals['number_of_runs'])]
    # save the number of trainable parameters
    globals['saved models'][name]['average trainable params'] = np.mean([model.number_of_trainable_parameters() for model in globals['saved models'][name]['models']])
    # save the losses
    losses = [torch.tensor(model.losses).detach().numpy() for model in globals['saved models'][name]['models']]
    globals['saved models'][name]['average losses'] = np.mean(losses, axis=0)
    globals['saved models'][name]['std losses'] = np.std(losses, axis=0)

In [ ]:
if globals['do_INN']:
    name = 'INN'
    _, ax = plt.subplots(nrows=3, figsize=(10, 20))

    ax[0].errorbar(range(len(globals['saved models'][name]['average losses'])), 
                   globals['saved models'][name]['average losses'], 
                   globals['saved models'][name]['std losses'],
                   capsize=5,
                   label=name)
    
    plot_model = globals['saved models'][name]['models'][0]
    I = torch.zeros(plot_model.model[0].in_features, plot_model.model[0].in_features)
    for i in range(plot_model.model[0].in_features):
        I[i, i] = 1.0
    M = plot_model.model[0].forward(I).detach().numpy()
    ax[1].spy(M, markersize=0.01)
    U,E,VT = np.linalg.svd(M)
    ax[2].plot(E)
    globals['saved models'][name]['singular values'] = E


# Random INN example that is same size as MLP

$$ 
\begin{bmatrix}
x & 0 & 0 & 0 
\end{bmatrix}
\begin{bmatrix}
I & S & S & S \\
0 & S & S & S \\
0 & S & S & S \\
0 & S & S & S
\end{bmatrix}
$$

In [ ]:
def train_sparse_INN():    
    data = INN_data_loader()
    default_module = 'SparseLinear.from_singleBlock'
    default_module_kwargs = {'block_type': 'R=0.4', 'initialization_type': f'G={globals["initialization mean"]},{globals["initialization std"]}', 'bias': globals["bias"]}
    # default_module = 'MaskedLinear.from_description'
    # default_module = 'SparseLinear.from_description'
    # default_module_kwargs = {'block_type': 'R=0.6', 'initialization_type': f'G={globals["initialization mean"]},{globals["initialization std"]}', 'trainable': 'non-zero', 'bias': globals["bias"]}

    cfg = {
        "model_type": "sequential2D",   
        "in_features_list": globals['layer_sizes'], 
        "out_features_list": globals['layer_sizes'], 

        "block_types": [
            ['Identity', default_module, default_module, default_module],
            [None,       default_module, default_module, default_module],
            [None,       default_module, default_module, default_module],
            [None,       default_module, default_module, default_module],
        ],
        "block_kwargs": [
            [None, default_module_kwargs, default_module_kwargs, default_module_kwargs],
            [None, default_module_kwargs, default_module_kwargs, default_module_kwargs],
            [None, default_module_kwargs, default_module_kwargs, default_module_kwargs],
            [None, default_module_kwargs, default_module_kwargs, default_module_kwargs],
        ],

        "activations" : ['Identity', 'ELU', 'ELU', 'Identity'],
        "activation_sizes" : globals['layer_sizes'],
        "callbacks": {
            "loss": {
                "func": "CrossEntropyLoss",
                "idx_list" : range(sum(globals['layer_sizes'][:-1]), sum(globals['layer_sizes'])),
            },
            "initialization": {
                "func": "zeros",
                "size": sum(globals['layer_sizes']),
            },
            "data": {
                "func": "insert",
                "idx_list": range(0, globals['layer_sizes'][0]),
                "flatten_input": True,
            },
            "output": {
                "func": "max",
                "idx_list" : range(sum(globals['layer_sizes'][:-1]), sum(globals['layer_sizes']))
            },
        }
    }
    # This is an IteratedModule, which is a wrapper around a module
    module = ModelFactory_callback(cfg)
    INN_module = IteratedModel(module.model, module.callbacks, normalize_loss=False, 
                               learning_rate=globals['learning rate'], optimizer=globals['optimizer'])
    trainer = pl.Trainer(max_epochs=globals['epochs'],
                         logger=LoggerFactory(),
                         devices=1 if torch.cuda.is_available() else 1,
                         accelerator=globals['accelerator'])
    trainer.fit(INN_module, data)
    return INN_module

In [ ]:
if globals['do_sparse_INN']:
    name = 'sparse INN'
    globals['saved models'][name] = {}
    # run all the models and save them
    globals['saved models'][name]['models'] = [train_sparse_INN() for i in range(globals['number_of_runs'])]
    # save the number of trainable parameters
    globals['saved models'][name]['average trainable params'] = np.mean([model.number_of_trainable_parameters() for model in globals['saved models'][name]['models']])
    # save the losses
    losses = [torch.tensor(model.losses).detach().numpy() for model in globals['saved models'][name]['models']]
    globals['saved models'][name]['average losses'] = np.mean(losses, axis=0)
    globals['saved models'][name]['std losses'] = np.std(losses, axis=0)

In [ ]:
if globals['do_sparse_INN']:
    name = 'sparse INN'
    _, ax = plt.subplots(nrows=3, figsize=(10, 20))

    ax[0].errorbar(range(len(globals['saved models'][name]['average losses'])), 
                   globals['saved models'][name]['average losses'], 
                   globals['saved models'][name]['std losses'],
                   capsize=5,
                   label=name)
    
    plot_model = globals['saved models'][name]['models'][0]
    I = torch.zeros(plot_model.model[0].in_features, plot_model.model[0].in_features)
    for i in range(plot_model.model[0].in_features):
        I[i, i] = 1.0
    M = plot_model.model[0].forward(I).detach().numpy()
    ax[1].spy(M, markersize=0.01)
    U,E,VT = np.linalg.svd(M)
    ax[2].plot(E)
    globals['saved models'][name]['singular values'] = E

# Summary

In [ ]:

for name in globals['saved models']:
    plt.errorbar(range(len(globals['saved models'][name]['average losses'])), 
                globals['saved models'][name]['average losses'], 
                globals['saved models'][name]['std losses'],
                capsize=5,
                label=name)
    plt.legend()

In [ ]:
for name in globals['saved models']:
    total_parameters = globals['saved models'][name]['average trainable params']
    average_loss = globals['saved models'][name]['average losses']
    print(f'{name} has {total_parameters} trainable parameters and an average loss of {average_loss[-1]}')

In [ ]:
for name in globals['saved models']:
    if 'singular values' in globals['saved models'][name].keys():
        plt.semilogy(globals['saved models'][name]['singular values'][:1000], label=name)
        plt.legend()